Import một số thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import implicit
import scipy.sparse as sparse
import torch
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

d:\Lab\DTU_RS\DTU_RS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

get_device()

device(type='cuda', index=0)

Kết nối đến MongoDB

In [3]:
uri = "mongodb+srv://<user>:<pass>@cluster0.jmil5cr.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"  # Thay user, pass của mình vào

# Kết nối tới server
client = MongoClient(uri, server_api=ServerApi('1'))

# Kiểm tra xem kết nối đã thành công chưa
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


Kết nối đến CSDL

In [4]:
db = client['dtu']

In [5]:
history = db["answered_questions"] # Collection chứa 100 câu hỏi người chơi đã chơi gần nhất

Dưới đây là Pipeline nhằm lấy ra dữ liệu theo định dạng mong muốn, hiện tại là bảng lịch sử 100 lần gần nhất đã được lấy ra từ DB (theo một cách nào đó... do chưa chốt DB cuối cùng). Ta thực hiện bóc tách các mảng và các thuộc tính của đối tượng

In [6]:
pipeline = [{"$unwind": "$questions"}, 
            {"$project": {"_id": 0, 
                          "player": "$playerId._id", 
                          "question": "$questions._id", 
                          "player_major": "$playerId.major", 
                          "player_rank": "$playerId.rank", 
                          "question_diff": "$questions.difficulty",
                          "question_category": "$questions.category",
                          "time": "$questions.timeForAnswer",
                          "status": "$questions.status"}}]

In [7]:
# Lấy dữ liệu từ DB
data = history.aggregate(pipeline)

Nhằm mục đích dễ phân tích, em sẽ chuyển dữ liệu lấy về thành Dataframe sử dụng Pandas

In [8]:
df = pd.DataFrame(list(history.aggregate(pipeline)))

In [9]:
df.head()

,player,question,player_major,player_rank,question_diff,question_category,time,status
0,65fbfc409a31efcf7a3fb085,65fbf56a4dba71a085a1e31b,"[Physics, Math, Eng, His]",3,4,Eng,6,0
1,65fbfc409a31efcf7a3fb085,65fbf56a4dba71a085a1d4f8,"[Physics, Math, Eng, His]",3,5,His,12,1
2,65fbfc409a31efcf7a3fb085,65fbfb83b5440169b33e087b,"[Physics, Math, Eng, His]",3,4,His,26,0
3,65fbfc409a31efcf7a3fb085,65fbfb83b5440169b33dfb68,"[Physics, Math, Eng, His]",3,3,Physics,27,1
4,65fbfc409a31efcf7a3fb085,65fbfb83b5440169b33e0a1c,"[Physics, Math, Eng, His]",3,4,His,23,0


Ở đây mỗi người chơi đều trả lời 100 câu hỏi, do đó ta sẽ chỉ loại bỏ các câu hỏi có ít hơn n (giả sử n = 2) người chơi:

In [10]:
df_players_per_question = (
    df.groupby(["question"]).agg({"player": "nunique"}).reset_index()
)
df_players_per_question.columns = ["question", "num_of_players"]

In [11]:
# Lấy ra danh sách các câu hỏi có nhiều hơn n người trả lời (giả sử n = 2)
num_of_players_threshold = 2

mask = df_players_per_question["num_of_players"] >= num_of_players_threshold
valid_questions = set(df_players_per_question.loc[mask, "question"].tolist())

In [12]:
# Lọc các record không phù hợp
df_filter_ques = df[df["question"].isin(valid_questions)].copy()

In [13]:
df.shape

(1000000, 8)

In [14]:
df_filter_ques.shape

(1000000, 8)

Lượng dữ liệu sau lọc không nhau mấy, do lượng câu hỏi được phân bổ khá đều

In [15]:
df_players_per_question["num_of_players"].min()

22

## Áp dụng Collaborative Filtering với thư viện Implicit

In [16]:
unique_players = df_filter_ques["player"].unique()
player_ids = dict(
    zip(unique_players, np.arange(unique_players.shape[0])))

unique_questions = df_filter_ques["question"].unique()
question_ids = dict(
    zip(unique_questions, np.arange(unique_questions.shape[0])))

df_filter_ques["player_id"] = df_filter_ques["player"].apply(
    lambda i: player_ids[i]
)
df_filter_ques["question_id"] = df_filter_ques["question"].apply(
    lambda i: question_ids[i]
)

In [17]:
print("Số người chơi: ", len(player_ids))
print("Số câu hỏi: ", len(question_ids))

Số người chơi:  6349
Số câu hỏi:  20000


In [18]:
# Lấy ra lĩnh vực của người chơi
unique_majors = df_filter_ques["player_major"].explode().unique()
unique_majors

array(['Physics', 'Math', 'Eng', 'His', 'Geo', 'Literature'], dtype=object)

In [19]:
# Lấy ra lĩnh vực của câu hỏi
unique_categories = df_filter_ques['question_category'].unique()
unique_categories

array(['Eng', 'His', 'Physics', 'Math', 'Literature', 'Geo'], dtype=object)

Để thực hiện Collaborative Filtering, cần tính giá trị $rating$ thể hiện cho độ phù hợp của người chơi với câu hỏi. Hiện tại công thức được đề xuất như sau:

$$rating = 0.2 \cdot performance + 0.3 \cdot sim(player \_ rank, question \_ diff) + 0.5 \cdot sim(player \_ major, question \_ category)$$

Trong đó: $performance = 1 - \frac{time}{max \_ time}$

Ta sẽ lần lượt tính các giá trị trong công thức trên

In [20]:
performance = 1 - df_filter_ques["time"]/60  # Giả sử thời gian tối đa là 60

In [21]:
# Áp dụng one hot encoding cho cột player_major và question_category
encoded_player_major = pd.get_dummies(df_filter_ques["player_major"].explode())
encoded_player_major = encoded_player_major.groupby(encoded_player_major.index).sum()

encoded_question_category = pd.get_dummies(df_filter_ques["question_category"])

# Tính cosine similarity giữa hai cột sau khi one-hot encoding
sim_player_question = torch.nn.functional.cosine_similarity(torch.tensor(encoded_player_major.values.astype(np.float32)).to(get_device()), 
                                                            torch.tensor(encoded_question_category.values.astype(np.float32)).to(get_device()))

In [22]:
# Tính similarity giữa player_rank (0-9) và question_diff (1-5)
rank_norm = df_filter_ques["player_rank"].apply(lambda x: x/9)
diff_norm = df_filter_ques["question_diff"].apply(lambda x: (x-1)/4)
max_values = pd.concat([rank_norm, diff_norm], axis=1).max(axis=1)

sim_rank_diff = (rank_norm - diff_norm).abs()/max_values

In [23]:
# Tính các giá trị rating

rating = (0.2 * performance) + (0.3 * sim_rank_diff) + (0.5 * sim_player_question.cpu().numpy())

In [24]:
rating.name = "rating"

In [25]:
df_player_ques_rating = pd.concat([df_filter_ques[["player_id", "question_id"]], rating], axis=1)

Ở đây vẫn còn một vấn đề trong data, có trường hợp người chơi trả lời lại một câu hỏi đã trả lời trước đó. Ta sẽ gộp lại và lấy mean rating.

In [26]:
df_player_ques_rating = df_player_ques_rating.groupby(["player_id", "question_id"]).agg({"rating": "mean"}).reset_index()

In [27]:
df_player_ques_rating.head()

,player_id,question_id,rating
0,0,0,0.596667
1,0,1,0.610000
2,0,2,0.530000
3,0,3,0.460000
4,0,4,0.540000


Bây giờ ta bắt đầu xây dựng mô hình dự đoán

In [28]:
#Tạo ma trận thưa

sparse_player_ques = sparse.csr_matrix(
    (
        df_player_ques_rating["rating"].astype(float),
        (df_player_ques_rating["player_id"], df_player_ques_rating["question_id"]),
    )
)

In [29]:
model = implicit.als.AlternatingLeastSquares()

d:\Lab\DTU_RS\DTU_RS\.venv\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [30]:
model.fit(sparse_player_ques)

100%|██████████| 15/15 [00:11<00:00,  1.36it/s]


## Thử sinh gợi ý

Đối với CF, có hai kết quả có thể được sinh ra từ model:

- Tìm các câu hỏi giống nhau

- Gợi ý câu hỏi cho người chơi

Ở đây ta sẽ chỉ quan tâm đến việc gợi ý câu hỏi cho người chơi.

In [31]:
df_player_id_map = df_filter_ques[["player", "player_id", "player_major"]].drop_duplicates(
    subset="player_id"
)

In [32]:
df_player_id_map.head()

,player,player_id,player_major
0,65fbfc409a31efcf7a3fb085,0,"[Physics, Math, Eng, His]"
100,65fbfc409a31efcf7a3f9e6d,1,"[His, Physics, Geo, Literature]"
200,65fbfc409a31efcf7a3f992c,2,"[Math, Literature, Geo, His]"
300,65fbfc409a31efcf7a3fa604,3,"[Eng, Literature, Math, Geo]"
400,65fbfc409a31efcf7a3fb03a,4,"[Math, Literature, Physics, Geo]"


In [33]:
df_ques_desc = df_filter_ques[
    ["question_id", "question", "question_category"]
].drop_duplicates(subset=["question_id"])

In [34]:
from bson import ObjectId

player = ObjectId("65fbfc409a31efcf7a3fb106") # Ta sẽ gợi ý cho người chơi có ID sau
player_id = df_player_id_map[df_player_id_map["player"] == player]["player_id"].item()
print(player_id)

1006


In [35]:
ids, scores = model.recommend(
    player_id, sparse_player_ques[player_id], N=50, filter_already_liked_items=True
)

In [36]:
list_questions = df_ques_desc[df_ques_desc["question_id"].isin(ids)]["question"].tolist()
list_desc = df_ques_desc[df_ques_desc["question_id"].isin(ids)]["question_category"].tolist()
df_recommendations = pd.DataFrame(
    {
        "question": list_questions,
        "question_category": list_desc,
        "score": scores,
        "already_liked": np.in1d(ids, sparse_player_ques[player_id].indices),
    }
)

In [37]:
df_recommendations

,question,question_category,score,already_liked
0,65fbfb83b5440169b33e0411,Eng,0.135988,False
1,65fbfb83b5440169b33dff4c,His,0.122309,False
2,65fbf56a4dba71a085a1e637,Literature,0.120429,False
3,65fbfb83b5440169b33e034f,His,0.107942,False
4,65fbf56a4dba71a085a1f789,His,0.107368,False
5,65fbf56a4dba71a085a1ec1c,Geo,0.106968,False
6,65fbfb83b5440169b33df761,Geo,0.105370,False
7,65fbfb83b5440169b33e02bf,Eng,0.104292,False
8,65fbfb83b5440169b33e02ca,Geo,0.103083,False
9,65fbfb83b5440169b33e04dc,Geo,0.101728,False


In [38]:
df_recommendations["question_category"].unique()

array(['Eng', 'His', 'Literature', 'Geo'], dtype=object)

In [39]:
df_player_id_map[df_player_id_map["player"] == player]["player_major"].item()

['Eng', 'Geo', 'Literature', 'His']

In [40]:
model.save("test.npz")